In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pickle

with open('/content/drive/MyDrive/egu23/data/covariates.pkl', 'rb') as f:
    covariates = pickle.load(f)

with open('/content/drive/MyDrive/egu23/data/target.pkl', 'rb') as f:
    target = pickle.load(f)

In [7]:
!pip install darts

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.9/693.9 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.0/100.0 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.4/572.4 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 17.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 718.6/718.6 kB 41.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 MB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46

In [8]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys


import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)


from darts import TimeSeries, concatenate
from darts.dataprocessing.transformers import Scaler
from darts.models import TransformerModel
from darts.metrics import mape, rmse, r2_score
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.utils.likelihood_models import QuantileRegression


pd.set_option("display.precision",2)
np.set_printoptions(precision=2, suppress=True)
pd.options.display.float_format = '{:,.2f}'.format

In [9]:
len(covariates)

15

In [10]:
i = 0
target_list = []
covar_list = []

for i in range(0, len(covariates)):
  if len(covariates[i]) < 1000:
    i +=1
  
  else:
    target_list.append(TimeSeries.from_dataframe(target[i], time_col = 'TIMESTAMP', freq = '1D'))
    covar_list.append(TimeSeries.from_dataframe(covariates[i], time_col = 'TIMESTAMP', freq = '1D'))
    

In [11]:
len(covar_list)

14

In [13]:
EPOCHS = 50
INLEN = 365          # input size
FEAT =  8          # d_model = number of expected features in the inputs, up to 512    
HEADS = 4           # default 8
ENCODE = 4          # encoder layers
DECODE = 4          # decoder layers
DIM_FF = 128        # dimensions of the feedforward network, default 2048
BATCH =  32        #1095          # batch size
ACTF = "relu"       # activation function, relu (default) or gelu
SCHLEARN = None     # a PyTorch learning rate scheduler; None = constant rate
LEARN = 1e-3        # learning rate
VALWAIT = 1         # epochs to wait before evaluating the loss on the test/validation set
DROPOUT = 0.1       # dropout rate
N_FC = 1            # output size

RAND = 42           # random seed
N_SAMPLES = 100     # number of times a prediction is sampled from a probabilistic model
N_JOBS = -1          # parallel processors to use;  -1 = all processors



SPLIT = 0.8         # train/test %

FIGSIZE = (9, 6)

In [14]:
NEE_train_list = []
NEE_test_list = [] 

for i in range(0, len(covar_list)):
  NEE_train, NEE_test = target_list[i].split_after(SPLIT)

  NEE_train = NEE_train.astype('float64')
  NEE_test = NEE_test.astype('float64')

  NEE_train_list.append(NEE_train)
  NEE_test_list.append(NEE_test)


In [16]:
scalerNEE = Scaler()
NEE_trtrain_list = []
NEE_trtest_list = []
NEE_tr_list = []

for i in range(0, len(covar_list)):

  scalerNEE.fit_transform(NEE_train_list[i])
  NEE_trtrain = scalerNEE.transform(NEE_train_list[i])
  NEE_trtest = scalerNEE.transform(NEE_test_list[i])
  NEE_tr = scalerNEE.transform(target_list[i])

  
  NEE_tr = NEE_tr.astype(np.float64)
  NEE_trtrain = NEE_trtrain.astype(np.float64)
  NEE_trtest = NEE_trtest.astype(np.float64)

  NEE_trtrain_list.append(NEE_trtrain)
  NEE_trtest_list.append(NEE_trtest)
  NEE_tr_list.append(NEE_tr)

# print("first and last row of scaled price time series:")
# pd.options.display.float_format = '{:,.2f}'.format
# NEE_tr.pd_dataframe().iloc[[0,-1]]

In [18]:
NEE_tr_list[0]

<TimeSeries (DataArray) (TIMESTAMP: 6209, component: 1, sample: 1)>
array([[[0.76]],

       [[0.74]],

       [[0.77]],

       ...,

       [[0.7 ]],

       [[0.67]],

       [[0.74]]])
Coordinates:
  * TIMESTAMP  (TIMESTAMP) datetime64[ns] 2003-01-01 2003-01-02 ... 2019-12-31
  * component  (component) object 'NEE_VUT_REF'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

In [19]:
covar_train_list = []
covar_test_list = [] 

for i in range(0, len(covar_list)):
  covar_train, covar_test = covar_list[i].split_after(SPLIT)

  covar_train = covar_train.astype('float64')
  covar_test = covar_test.astype('float64')


  covar_train_list.append(covar_train)
  covar_test_list.append(covar_test)

In [20]:
scalerNEE = Scaler()
covar_trtrain_list = []
covar_trtest_list = []
covar_tr_list = []

for i in range(0, len(covar_list)):

  scalerNEE.fit_transform(covar_train_list[i])
  covar_trtrain = scalerNEE.transform(covar_train_list[i])
  covar_trtest = scalerNEE.transform(covar_test_list[i])
  covar_tr = scalerNEE.transform(covar_list[i])

  
  covar_tr = covar_tr.astype(np.float64)
  covar_trtrain = covar_trtrain.astype(np.float64)
  covar_trtest = covar_trtest.astype(np.float64)

  covar_trtrain_list.append(covar_trtrain)
  covar_trtest_list.append(covar_trtest)
  covar_tr_list.append(covar_tr)


In [ ]:
covar_tr_list[0]

In [23]:
# default quantiles for QuantileRegression
QUANTILES = [0.01, 0.1, 0.2, 0.5, 0.8, 0.9, 0.99]

In [28]:
model = TransformerModel(
                    input_chunk_length = INLEN,
                    output_chunk_length = N_FC,
                    batch_size = BATCH,
                    n_epochs = EPOCHS,
                    model_name = "Transformer_NEE",
                    nr_epochs_val_period = VALWAIT,
                    d_model = FEAT,
                    nhead = HEADS,
                    num_encoder_layers = ENCODE,
                    num_decoder_layers = DECODE,
                    dim_feedforward = DIM_FF,
                    dropout = DROPOUT,
                    activation = ACTF,
                    random_state=RAND,
                    likelihood=QuantileRegression(quantiles=QUANTILES), 
                    optimizer_kwargs={'lr': LEARN},
                    add_encoders={"cyclic": {"future": ["hour", 'month']}},
                    save_checkpoints=True,
                    force_reset=True
                    )

In [29]:
model.fit(NEE_trtrain_list, past_covariates=covar_tr_list, verbose=True)

Training: 0it [00:00, ?it/s]

TransformerModel(d_model=8, nhead=4, num_encoder_layers=4, num_decoder_layers=4, dim_feedforward=128, dropout=0.1, activation=relu, norm_type=None, custom_encoder=None, custom_decoder=None, input_chunk_length=365, output_chunk_length=1, batch_size=32, n_epochs=10, model_name=Transformer_NEE, nr_epochs_val_period=1, random_state=42, likelihood=<darts.utils.likelihood_models.QuantileRegression object at 0x7fa14ae2ca30>, optimizer_kwargs={'lr': 0.001}, add_encoders={'cyclic': {'future': ['hour', 'month']}}, save_checkpoints=True, force_reset=True)

In [1]:
# testing: generate predictions
pred_list = []

for i in range(0, len(NEE_trtest_list)):
  NEE_pred = model.predict(n=len(NEE_trtest_list[i]),
                           series = NEE_train_list[i],
                           past_covariates=covar_list[i], 
                           num_samples=N_SAMPLES,   
                           n_jobs=N_JOBS, 
                           verbose=True)
  
  pred_list.append(NEE_pred)

import pickle

with open('./predictions.pkl', 'wb') as f:
  pickle.dump(pred_list, f)

with open('./actual.pkl', 'wb') as f:
  pickle.dump(NEE_test_list, f)

NameError: ignored

In [ ]:
for i in range(len(pred_list)):
  RMSE = rmse(NEE_test_list[i], pred_list[i])
  MAPE = mape(NEE_test_list[i], pred_list[i])
  R2 = r2_score(NEE_test_list[i], pred_list[i])
  
  print("RMSE:", f'{RMSE:.2f}')
  print("MAPE:", f'{MAPE:.2f}')
  print("R2:", f'{R2:.2f}')


In [ ]:
NEE_test_list[1].plot()
pred_list[1].plot()